In [16]:
import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt
import pandas as pd

In [17]:
for i in range(100):
    face_path = './dataset/train/img/%s.png'%i
    face_origin = cv2.imread(face_path)
    face_copy = face_origin.copy()
    face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    detector = dlib.get_frontal_face_detector()
    dlib_rects = detector(face_rgb, 1)
    
    img_how_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    globals()['height'+str(i)] = face_origin.shape[0]
    
    globals()['img'+str(i)] = []
    
    for dlib_rect in dlib_rects:
        points = predictor(face_rgb, dlib_rect)
        list_points = list(map(lambda p: [p.x, p.y], points.parts()))
        globals()['img'+str(i)].append(list_points)
        globals()['img'+str(i)] = np.array(globals()['img'+str(i)])

In [3]:
for i in range(100):
    land = np.zeros((68,2))
    
    for j in range(68):
        land[j][0] = globals()['img'+str(i)][0][j][0]
        land[j][1] = globals()['img'+str(i)][0][j][1]
        
    x = land[:,0]
    y = land[:,1] * (-1) + globals()['height'+str(i)]
    
    x -= ((min(x) + max(x)) / 2)
    y -= ((min(y) + max(y)) / 2)
    
    norm_x = (x - min(x)) / ((max(y)-min(y)))
    norm_y = (y - min(y)) / ((max(y)-min(y)))
    
    globals()['norm'+str(i)] = np.column_stack((norm_x, norm_y))

In [4]:
# xyxyxyxyxyxyxy(36~47)
for i in range(100):
    globals()['norm'+str(i)] = globals()['norm'+str(i)][36:48]
    globals()['norm'+str(i)] = globals()['norm'+str(i)].flatten()

In [5]:
train_x = np.zeros((100, 24))
for i in range(100):
    for j in range(24):
        train_x[i][j] = globals()['norm'+str(i)][j]   

In [6]:
for i in range(100):
    face_path = './dataset/test/img/%s.png'%i
    face_origin = cv2.imread(face_path)
    face_copy = face_origin.copy()
    face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    detector = dlib.get_frontal_face_detector()
    dlib_rects = detector(face_rgb, 1)
    
    img_how_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    globals()['height'+str(i)] = face_origin.shape[0]
    
    globals()['img'+str(i)] = []
    
    for dlib_rect in dlib_rects:
        points = predictor(face_rgb, dlib_rect)
        list_points = list(map(lambda p: [p.x, p.y], points.parts()))
        globals()['img'+str(i)].append(list_points)
        globals()['img'+str(i)] = np.array(globals()['img'+str(i)])

In [7]:
for i in range(100):
    land = np.zeros((68,2))
    
    for j in range(68):
        land[j][0] = globals()['img'+str(i)][0][j][0]
        land[j][1] = globals()['img'+str(i)][0][j][1]
        
    x = land[:,0]
    y = land[:,1] * (-1) + globals()['height'+str(i)]
    
    x -= ((min(x) + max(x)) / 2)
    y -= ((min(y) + max(y)) / 2)
    
    norm_x = (x - min(x)) / (max(y)-min(y))
    norm_y = (y - min(y)) / (max(y)-min(y))
    
    globals()['norm'+str(i)] = np.column_stack((norm_x, norm_y))

In [8]:
# xyxyxyxyxyxyxy(36~47)
for i in range(100):
    globals()['norm'+str(i)] = globals()['norm'+str(i)][36:48]
    globals()['norm'+str(i)] = globals()['norm'+str(i)].flatten()

In [9]:
test_x = np.zeros((100, 24))
for i in range(100):
    for j in range(24):
        test_x[i][j] = globals()['norm'+str(i)][j]   

In [10]:
train_y = pd.read_csv('./dataset/train/train.csv')
train_y = train_y.iloc[:, 16:22]
test_y = pd.read_csv('./dataset/test/test.csv')
test_y = test_y.iloc[:, 16:22]

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

model = LinearRegression()
model.fit(train_x, train_y)
y_pred = model.predict(train_x)
mae = mean_absolute_error(train_y, y_pred)

mae

0.0734935297091187

In [12]:
y_pred = model.predict(test_x)
mae = mean_absolute_error(test_y, y_pred)

mae

0.11068429731796842

In [15]:
# model.intercept_, model.coef_

In [13]:
prediction = pd.DataFrame(y_pred)
prediction.columns = ['eyeSize','eyePosition', 'eyeDepth','eyeRotation','eyeDistance','eyeSquint']
prediction

,eyeSize,eyePosition,eyeDepth,eyeRotation,eyeDistance,eyeSquint
0,0.353146,0.638937,0.621232,0.490727,0.414744,0.458249
1,0.327979,0.398876,0.750656,0.627314,0.440319,0.582535
2,0.548795,0.492176,0.380265,0.638749,0.307966,0.591691
3,0.550444,0.490864,0.545807,0.357172,0.397181,0.403463
4,0.639292,0.585837,0.553815,0.368740,0.499729,0.586210
...,...,...,...,...,...,...
95,0.705320,0.514965,0.420613,0.383593,0.281383,0.612815
96,0.795361,0.371456,0.723106,0.658041,0.552763,0.610311
97,0.477682,0.686095,0.610237,0.367162,0.566539,0.466785
98,0.441157,0.349790,0.536356,0.435280,0.603896,0.493593


In [14]:
test_y

,eyeSize,eyePosition,eyeDepth,eyeRotation,eyeDistance,eyeSquint
0,0.439447,0.758647,0.370812,0.506665,0.462228,0.754029
1,0.235531,0.306835,0.310255,0.799276,0.280601,0.490777
2,0.728159,0.711342,0.489252,0.768084,0.467365,0.736168
3,0.647029,0.614792,0.624584,0.250405,0.448013,0.523622
4,0.763835,0.662353,0.226584,0.220240,0.569739,0.638667
...,...,...,...,...,...,...
95,0.747057,0.606980,0.688712,0.295096,0.276458,0.663174
96,0.757262,0.391804,0.574604,0.728450,0.522360,0.746065
97,0.345698,0.455230,0.572820,0.659950,0.482190,0.345089
98,0.334341,0.318302,0.478478,0.285375,0.471584,0.323365


In [15]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(test_y, prediction)**0.5
print(rmse)

0.14232892317052015
